In [2]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [10]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [11]:
def conv2d(x,W):
    # x --> [Batch,H,W,Channels]
    # W --> [filter H, filter W, Channels IN, Channels OUT]
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [12]:
def max_pool_2by2(x):
    # x --> [batch, h, w, c]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [14]:
# CONV Layer
def convelutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [37]:
# Normal LAyer
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b

In [38]:
# Placeholder
x = tf.placeholder(tf.float32, shape=[None,784])

In [39]:
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [40]:
# layers
x_image = tf.reshape(x, [-1,28,28,1])

In [41]:
convo_1 = convelutional_layer(x_image, shape=[5,5,1,32]) 
# 5x5 is potach size, 1 is input channel, 32 is freatures (outputs)
convo_1_pooling = max_pool_2by2(convo_1)

In [42]:
convo_2 = convelutional_layer(convo_1_pooling, shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [43]:
convo_2_flat = tf.reshape(convo_2_pooling, [-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [44]:
# Dropout

hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [45]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [46]:
# LOSS Fucntion
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [47]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [53]:
init = tf.global_variables_initializer()

In [54]:
steps = 1000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prob:0.5})
        if i%100 == 0:
            
            print("ON step: {}".format(i))
            print("Accuracy: ")
            
            matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            print(sess.run(acc, feed_dict={x:mnist.test.images,y_true:mnist.test.labels, hold_prob:1.0}))

ON step: 0
Accuracy: 
0.138
ON step: 100
Accuracy: 
0.9477
ON step: 200
Accuracy: 
0.9634
ON step: 300
Accuracy: 
0.9659
ON step: 400
Accuracy: 
0.975
ON step: 500
Accuracy: 
0.9788
ON step: 600
Accuracy: 
0.9767
ON step: 700
Accuracy: 
0.9817
ON step: 800
Accuracy: 
0.9821
ON step: 900
Accuracy: 
0.979
